In [182]:
import pandas as pd
#pd.set_option('display.max_colwidth', None)
import googlemaps
import requests
import time
import json
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium import plugins
import config

api_key = config.api_key

In [183]:
zipcodes_geo = gpd.read_file('/Users/k.simmons/Desktop/NSS/data analytics bootcamp/capstone/Data/zipcodes.geojson')

In [184]:
zipcodes_geo.head()

zip objectid    po_name      shape_stlength        shape_starea  \
0  37115        1    MADISON  178783.02488886821  596553400.57885742   
1  37216        3  NASHVILLE  75820.997821400058  188884682.28344727   
2  37204        9  NASHVILLE  93180.292250425613  200664795.51708984   
3  37027       11  BRENTWOOD   159760.6942933173  174978422.04101562   
4  37064       18   FRANKLIN  28995.828320601937  46969608.005737305   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# creating list of zipcodes that are contained within Nashville city limits (not entire Metro Nashville-Davidson County)

In [185]:
nashmask = zipcodes_geo['po_name'].isin(['NASHVILLE'])

In [186]:
nashzips = zipcodes_geo[nashmask].reset_index(drop=True)

37207 is listed twice in the geodata, because the zipcode is split geographically. To avoid key:value issues later on, I am eliminating the smaller 37207 polygon.

In [187]:
drop49 = nashzips['objectid'].isin(['49'])

In [188]:
nashzips[drop49]

zip objectid    po_name      shape_stlength        shape_starea  \
7  37207       49  NASHVILLE  14456.130686344946  6987521.0933837891   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [189]:
nashzips = nashzips.drop(index=7)

In [190]:
nashzips['zip']

0     37216
1     37204
2     37221
3     37212
4     37209
5     37206
6     37217
8     37219
9     37218
10    37232
11    37213
12    37205
13    37203
14    37201
15    37214
16    37240
17    37208
18    37207
19    37215
20    37228
21    37210
22    37211
23    37220
Name: zip, dtype: object

doublechecking that I only retained the larger of the two polygons

In [194]:
nash37207 = nashzips['zip'].isin(['37207'])

In [196]:
nashzips[nash37207]

zip objectid    po_name      shape_stlength        shape_starea  \
18  37207       48  NASHVILLE  196775.60104082408  556772811.32653809   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [197]:
nashzips.to_excel('nashzipcodes.xlsx')

In [198]:
polygon37216 = nashzips.loc[nashzips['zip'] == '37216']
polygon37204 = nashzips.loc[nashzips['zip'] == '37204']
polygon37221 = nashzips.loc[nashzips['zip'] == '37221']
polygon37212 = nashzips.loc[nashzips['zip'] == '37212']
polygon37209 = nashzips.loc[nashzips['zip'] == '37209']
polygon37206 = nashzips.loc[nashzips['zip'] == '37206']
polygon37217 = nashzips.loc[nashzips['zip'] == '37217']
polygon37207 = nashzips.loc[nashzips['zip'] == '37207']
polygon37219 = nashzips.loc[nashzips['zip'] == '37219']
polygon37218 = nashzips.loc[nashzips['zip'] == '37218']
polygon37232 = nashzips.loc[nashzips['zip'] == '37232']
polygon37213 = nashzips.loc[nashzips['zip'] == '37213']
polygon37205 = nashzips.loc[nashzips['zip'] == '37205']
polygon37203 = nashzips.loc[nashzips['zip'] == '37203']
polygon37201 = nashzips.loc[nashzips['zip'] == '37201']
polygon37214 = nashzips.loc[nashzips['zip'] == '37214']
polygon37240 = nashzips.loc[nashzips['zip'] == '37240']
polygon37208 = nashzips.loc[nashzips['zip'] == '37208']
polygon37215 = nashzips.loc[nashzips['zip'] == '37215']
polygon37228 = nashzips.loc[nashzips['zip'] == '37228']
polygon37210 = nashzips.loc[nashzips['zip'] == '37210']
polygon37211 = nashzips.loc[nashzips['zip'] == '37211']
polygon37220 = nashzips.loc[nashzips['zip'] == '37220']

# creating centroid for each Nashville zipcode, to aid in Google API queries

In [200]:
center37216 = polygon37216.geometry.centroid
center37204 = polygon37204.geometry.centroid
center37221 = polygon37221.geometry.centroid
center37212 = polygon37212.geometry.centroid
center37209 = polygon37209.geometry.centroid
center37206 = polygon37206.geometry.centroid
center37217 = polygon37217.geometry.centroid
center37207 = polygon37207.geometry.centroid
center37219 = polygon37219.geometry.centroid
center37218 = polygon37218.geometry.centroid
center37232 = polygon37232.geometry.centroid
center37213 = polygon37213.geometry.centroid
center37205 = polygon37205.geometry.centroid
center37203 = polygon37203.geometry.centroid
center37201 = polygon37201.geometry.centroid
center37214 = polygon37214.geometry.centroid
center37240 = polygon37240.geometry.centroid
center37208 = polygon37208.geometry.centroid
center37215 = polygon37215.geometry.centroid
center37228 = polygon37228.geometry.centroid
center37210 = polygon37210.geometry.centroid
center37211 = polygon37211.geometry.centroid
center37220 = polygon37220.geometry.centroid

/var/folders/s3/87cx_4f97x52rgnl47c9skpr0000gn/T/ipykernel_5090/3432679964.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center37216 = polygon37216.geometry.centroid
/var/folders/s3/87cx_4f97x52rgnl47c9skpr0000gn/T/ipykernel_5090/3432679964.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center37204 = polygon37204.geometry.centroid
/var/folders/s3/87cx_4f97x52rgnl47c9skpr0000gn/T/ipykernel_5090/3432679964.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center37221 = polygon37221.geometry.centroid
/var/folders/s3/87cx_4f97x52rgnl47c9skpr0000gn/T/ipykernel_50

# creating list of LAT/LONG of each Zipcode's centroid, for inclusion in Google API query

In [203]:
center37207

18    POINT (-86.77677 36.23245)
dtype: geometry

In [202]:
print(center37216[0])

POINT (-86.7263455721062 36.21605012332708)


In [204]:
latlng37216 = f'{center37216[0].y},{center37216[0].x}'
latlng37204 = f'{center37204[1].y},{center37204[1].x}'
latlng37221 = f'{center37221[2].y},{center37221[2].x}'
latlng37212 = f'{center37212[3].y},{center37212[3].x}'
latlng37209 = f'{center37209[4].y},{center37209[4].x}'
latlng37206 = f'{center37206[5].y},{center37206[5].x}'
latlng37217 = f'{center37217[6].y},{center37217[6].x}'
latlng37219 = f'{center37219[8].y},{center37219[8].x}'
latlng37218 = f'{center37218[9].y},{center37218[9].x}'
latlng37232 = f'{center37232[10].y},{center37232[10].x}'
latlng37213 = f'{center37213[11].y},{center37213[11].x}'
latlng37205 = f'{center37205[12].y},{center37205[12].x}'
latlng37203 = f'{center37203[13].y},{center37203[13].x}'
latlng37201 = f'{center37201[14].y},{center37201[14].x}'
latlng37214 = f'{center37214[15].y},{center37214[15].x}'
latlng37240 = f'{center37240[16].y},{center37240[16].x}'
latlng37208 = f'{center37208[17].y},{center37208[17].x}'
latlng37207 = f'{center37207[18].y},{center37207[18].x}'
latlng37215 = f'{center37215[19].y},{center37215[19].x}'
latlng37228 = f'{center37228[20].y},{center37228[20].x}'
latlng37210 = f'{center37210[21].y},{center37210[21].x}'
latlng37211 = f'{center37211[22].y},{center37211[22].x}'
latlng37220 = f'{center37220[23].y},{center37220[23].x}'

In [205]:
latlng37207

'36.23245257407244,-86.77676762469858'

In [206]:
type(''.join(latlng37207))

str

In [207]:
type(latlng37207)

str

# made Nashville zipcode boundary map for my own visualization purposes while in workflow

In [208]:
nash_center37203 = center37203.geometry.centroid[13]

/var/folders/s3/87cx_4f97x52rgnl47c9skpr0000gn/T/ipykernel_5090/1914074261.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nash_center37203 = center37203.geometry.centroid[13]


In [209]:
print(nash_center37203)

POINT (-86.78898899667411 36.14860232841574)


In [210]:
nash_center = [nash_center37203.y, nash_center37203.x]
print(nash_center)

[36.14860232841574, -86.7889889966741]


In [212]:
nash_map = folium.Map(location = nash_center, zoom_start = 10)
plugins.MeasureControl().add_to(nash_map)

In [213]:
nash_map

In [214]:
folium.GeoJson(polygon37216).add_to(nash_map)
folium.GeoJson(polygon37204).add_to(nash_map)
folium.GeoJson(polygon37221).add_to(nash_map)
folium.GeoJson(polygon37212).add_to(nash_map)
folium.GeoJson(polygon37209).add_to(nash_map)
folium.GeoJson(polygon37206).add_to(nash_map)
folium.GeoJson(polygon37217).add_to(nash_map)
folium.GeoJson(polygon37207).add_to(nash_map)
folium.GeoJson(polygon37219).add_to(nash_map)
folium.GeoJson(polygon37218).add_to(nash_map)
folium.GeoJson(polygon37232).add_to(nash_map)
folium.GeoJson(polygon37213).add_to(nash_map)
folium.GeoJson(polygon37205).add_to(nash_map)
folium.GeoJson(polygon37203).add_to(nash_map)
folium.GeoJson(polygon37201).add_to(nash_map)
folium.GeoJson(polygon37214).add_to(nash_map)
folium.GeoJson(polygon37240).add_to(nash_map)
folium.GeoJson(polygon37208).add_to(nash_map)
folium.GeoJson(polygon37215).add_to(nash_map)
folium.GeoJson(polygon37228).add_to(nash_map)
folium.GeoJson(polygon37210).add_to(nash_map)
folium.GeoJson(polygon37211).add_to(nash_map)
folium.GeoJson(polygon37220).add_to(nash_map)


In [216]:
nash_map

# there are 23 zipcodes within Nashville city limits to do queries for

# beginning of queries for supermarkets/grocers using lat/lng centroids for each zipcode

37204 supermarkets/grocers, 2000 meter radius

In [ ]:
grocery_stores37204 = []
params37204 = {}

#endpoint_url37204 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery+store+in+37204&location=36.10566266613879,-86.7746655408201&radius=2000&region=us&type=grocery_or_supermarket,supermarket,bakery,food&key={api_key}"

res37204 = requests.get(endpoint_url37204, params = params37204)
results37204 = json.loads(res37204.content)
grocery_stores37204.extend(results37204['results'])
time.sleep(2)
while "next_page_token" in results37204:
    params37204['pagetoken'] = results37204['next_page_token'],
    res37204 = requests.get(endpoint_url37204, params = params37204)
    results37204 = json.loads(res37204.content)
    grocery_stores37204.extend(results37204['results'])
    time.sleep(2)

In [ ]:
res37204

In [ ]:
results37204

In [ ]:
shop_name37204 = []
formatted_address37204 = []
shop_lat37204 = []
shop_lng37204 = []

for i in range(len(grocery_stores37204)):
    shop37204 = grocery_stores37204[i]

    try:
        shop_name37204.append(shop37204['name'])
    except:
        shop_name37204.append('none')

    try:
        formatted_address37204.append(shop37204['formatted_address'])
    except:
        formatted_address37204.append('none')

    try:
        shop_lat37204.append(shop37204['geometry']['location']['lat'])
    except:
        shop_lat37204.append('none')

    try:
        shop_lng37204.append(shop37204['geometry']['location']['lng'])
    except:
        shop_lng37204.append('none')

In [ ]:
df_dict37204 = {'shop_name':shop_name37204, 'shop_address':formatted_address37204, 'shop_lat':shop_lat37204, 'shop_lng':shop_lng37204}

grocery_stores37204_df = pd.DataFrame(df_dict37204)

grocery_stores37204_df['duplicator'] = grocery_stores37204_df['shop_name'] + grocery_stores37204_df['shop_address']

grocery_stores37204_df.drop_duplicates(['duplicator'], inplace=True)

In [ ]:
grocery_stores37204_df

37221 supermarkets/grocers, 8000 meter radius

In [ ]:
grocery_stores37221 = []
params37221 = {}

#endpoint_url37221 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery+store+in+37221&location=36.06283153351842,-86.97531387608755&radius=8000&region=us&type=grocery_or_supermarket,supermarket,bakery,food&key={api_key}"

res37221 = requests.get(endpoint_url37221, params = params37221)
results37221 = json.loads(res37221.content)
grocery_stores37221.extend(results37221['results'])
time.sleep(2)
while "next_page_token" in results37221:
    params37221['pagetoken'] = results37221['next_page_token'],
    res37221 = requests.get(endpoint_url37221, params = params37221)
    results37221 = json.loads(res37221.content)
    grocery_stores37221.extend(results37221['results'])
    time.sleep(2)

In [ ]:
res37221

In [ ]:
results37221

In [ ]:
shop_name37221 = []
formatted_address37221 = []
shop_lat37221 = []
shop_lng37221 = []

for i in range(len(grocery_stores37221)):
    shop37221 = grocery_stores37221[i]

    try:
        shop_name37221.append(shop37221['name'])
    except:
        shop_name37221.append('none')

    try:
        formatted_address37221.append(shop37221['formatted_address'])
    except:
        formatted_address37221.append('none')

    try:
        shop_lat37221.append(shop37221['geometry']['location']['lat'])
    except:
        shop_lat37221.append('none')

    try:
        shop_lng37221.append(shop37221['geometry']['location']['lng'])
    except:
        shop_lng37221.append('none')

In [ ]:
df_dict37221 = {'shop_name':shop_name37221, 'shop_address':formatted_address37221, 'shop_lat':shop_lat37221, 'shop_lng':shop_lng37221}

grocery_stores37221_df = pd.DataFrame(df_dict37221)

grocery_stores37221_df['duplicator'] = grocery_stores37221_df['shop_name'] + grocery_stores37221_df['shop_address']

grocery_stores37221_df.drop_duplicates(['duplicator'], inplace=True)

In [ ]:
grocery_stores37221_df

37212 supermarkets/grocers, 8000 meter radius

In [ ]:
grocery_stores37212 = []
params37212 = {}

#endpoint_url37212 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery+store+in+37212&location=36.133345610650146,-86.80157366719409&radius=8000&region=us&type=grocery_or_supermarket,supermarket,bakery,food&key={api_key}"

res37212 = requests.get(endpoint_url37212, params = params37212)
results37212 = json.loads(res37212.content)
grocery_stores37212.extend(results37212['results'])
time.sleep(2)
while "next_page_token" in results37212:
    params37212['pagetoken'] = results37212['next_page_token'],
    res37212 = requests.get(endpoint_url37212, params = params37212)
    results37212 = json.loads(res37212.content)
    grocery_stores37212.extend(results37212['results'])
    time.sleep(2)

In [ ]:
res37212

In [ ]:
results37212

In [ ]:
shop_name37212 = []
formatted_address37212 = []
shop_lat37212 = []
shop_lng37212 = []

for i in range(len(grocery_stores37212)):
    shop37212 = grocery_stores37212[i]

    try:
        shop_name37212.append(shop37212['name'])
    except:
        shop_name37212.append('none')

    try:
        formatted_address37212.append(shop37212['formatted_address'])
    except:
        formatted_address37212.append('none')

    try:
        shop_lat37212.append(shop37212['geometry']['location']['lat'])
    except:
        shop_lat37212.append('none')

    try:
        shop_lng37212.append(shop37212['geometry']['location']['lng'])
    except:
        shop_lng37212.append('none')

In [ ]:
df_dict37212 = {'shop_name':shop_name37212, 'shop_address':formatted_address37212, 'shop_lat':shop_lat37212, 'shop_lng':shop_lng37212}

grocery_stores37212_df = pd.DataFrame(df_dict37212)

grocery_stores37212_df['duplicator'] = grocery_stores37212_df['shop_name'] + grocery_stores37212_df['shop_address']

grocery_stores37212_df.drop_duplicates(['duplicator'], inplace=True)

In [ ]:
grocery_stores37212_df

In [217]:
usedzips = nashzips['zip']
usedzips = usedzips.tolist()

In [218]:
usedzips

['37216',
 '37204',
 '37221',
 '37212',
 '37209',
 '37206',
 '37217',
 '37219',
 '37218',
 '37232',
 '37213',
 '37205',
 '37203',
 '37201',
 '37214',
 '37240',
 '37208',
 '37207',
 '37215',
 '37228',
 '37210',
 '37211',
 '37220']

# function dictionaries

creating dictionary with approximate measurement of longest width of zipcode, to be halved for Places API radius parameter.

In [220]:
zipradius = {'radius37216' : str(6424/2),          
             'radius37204' : str(4413/2),
             'radius37221' : str(14868/2),
             'radius37212' : str(3617/2),
             'radius37209' : str(16250/2),
             'radius37206' : str(7317/2),
             'radius37217' : str(12820/2),
             'radius37207' : str(12584/2),
             'radius37219' : str(1536/2),
             'radius37218' : str(15417/2),
             'radius37232' : str(803/2),
             'radius37213' : str(2961/2),
             'radius37205' : str(10009/2),
             'radius37203' : str(5827/2),
             'radius37201' : str(2193/2),
             'radius37214' : str(15907/2),
             'radius37240' : str(540/2),
             'radius37208' : str(5053/2),
             'radius37215' : str(8832/2),
             'radius37228' : str(3798/2),
             'radius37210' : str(6575/2),
             'radius37211' : str(14141/2),
             'radius37220' : str(6084/2),
           }

finally, now using the lists I made to create a dictionary of the centroid coordinates

In [221]:
coord_dict = {'coord37216' : ''.join(latlng37216),
              'coord37204' : ''.join(latlng37204),
              'coord37221' : ''.join(latlng37221),
              'coord37212' : ''.join(latlng37212),
              'coord37209' : ''.join(latlng37209),
              'coord37206' : ''.join(latlng37206),
              'coord37217' : ''.join(latlng37217),
              'coord37207' : ''.join(latlng37207),
              'coord37219' : ''.join(latlng37219),
              'coord37218' : ''.join(latlng37218),
              'coord37232' : ''.join(latlng37232),
              'coord37213' : ''.join(latlng37213),
              'coord37205' : ''.join(latlng37205),
              'coord37203' : ''.join(latlng37203),
              'coord37201' : ''.join(latlng37201),
              'coord37214' : ''.join(latlng37214),
              'coord37240' : ''.join(latlng37240),
              'coord37208' : ''.join(latlng37208),
              'coord37215' : ''.join(latlng37215),
              'coord37228' : ''.join(latlng37228),
              'coord37210' : ''.join(latlng37210),
              'coord37211' : ''.join(latlng37211),
              'coord37220' : ''.join(latlng37220),
             }

creating a combined dictionary from the prior two dictionaries

In [222]:
zip_codes = { int(zip_code.replace("radius", "")): { "radius": zipradius[zip_code], } for zip_code in zipradius }
for zip_code in zip_codes:
    zip_codes[zip_code]["coord"] = coord_dict[f"coord{zip_code}"]

In [223]:
len(zip_codes)

23

In [224]:
len(coord_dict)

23

In [225]:
len(zipradius)

23

# okay, manually changing the query code by copy/pasting the zipcode many times is much too tedious, drawn out, and prone to error. Time to call in the query function:

supermarket/grocery query function:

In [227]:
def grocery_query(zip, coord_dict, zipradius):
    grocery_stores = []

    params = { 
            "query": f"grocery+in+{zip}",
            "location": coord_dict,
            "address": f"zipcode {zip}",
            "radius": zipradius,
            "region": "us",
            "type": "grocery_or_supermarket,supermarket",
            "key": config.api_key,
            }

    endpoint_url= "https://maps.googleapis.com/maps/api/place/textsearch/json"

    res = requests.get(endpoint_url, params = params)
    results = res.json()
    grocery_stores.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        grocery_stores.extend(results['results'])
        time.sleep(2)

    shop_name = []
    formatted_address = []
    shop_lat = []
    shop_lng = []
    type = []

    for i in range(len(grocery_stores)):
        shop = grocery_stores[i]

        try:
            shop_name.append(shop['name'])
        except:
            shop_name.append('none')

        try:
            formatted_address.append(shop['formatted_address'])
        except:
            formatted_address.append('none')

        try:
            shop_lat.append(shop['geometry']['location']['lat'])
        except:
            shop_lat.append('none')

        try:
            shop_lng.append(shop['geometry']['location']['lng'])
        except:
            shop_lng.append('none')
        
        try:
            type.append(shop['types'])
        except:
            type.append('none')

    df_dict = {'shop_name':shop_name, 'shop_address':formatted_address, 'shop_lat':shop_lat, 'shop_lng':shop_lng, 'types':type}

    grocery_stores = pd.DataFrame(df_dict)

    grocery_stores.to_csv(f'Data/API_queries/grocery/grocery_stores{zip}.csv')

    return grocery_stores

    

gas station query function:

In [228]:
def gasstation_query(zip, coord_dict, zipradius):
    gas_stations= []

    params = { 
            "query": f"gas+station+in+{zip}",
            "location": coord_dict,
            "address": f"zipcode {zip}",
            "radius": zipradius,
            "region": "us",
            "type": "gas_station",
            "key": config.api_key,
            }

    endpoint_url= "https://maps.googleapis.com/maps/api/place/textsearch/json"

    res = requests.get(endpoint_url, params = params)
    results = res.json()
    gas_stations.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        gas_stations.extend(results['results'])
        time.sleep(2)

    shop_name = []
    formatted_address = []
    shop_lat = []
    shop_lng = []
    type = []

    for i in range(len(gas_stations)):
        shop = gas_stations[i]

        try:
            shop_name.append(shop['name'])
        except:
            shop_name.append('none')

        try:
            formatted_address.append(shop['formatted_address'])
        except:
            formatted_address.append('none')

        try:
            shop_lat.append(shop['geometry']['location']['lat'])
        except:
            shop_lat.append('none')

        try:
            shop_lng.append(shop['geometry']['location']['lng'])
        except:
            shop_lng.append('none')
        
        try:
            type.append(shop['types'])
        except:
            type.append('none')

    df_dict = {'shop_name':shop_name, 'shop_address':formatted_address, 'shop_lat':shop_lat, 'shop_lng':shop_lng, 'types':type}

    gas_stations = pd.DataFrame(df_dict)

    gas_stations.to_csv(f'Data/API_queries/gas_station/gas_stations{zip}.csv')

    return gas_stations


convenience store query

In [229]:
def convstore_query(zip, coord_dict, zipradius):
    convenience_store= []

    params = { 
            "query": f"convenience+store+in+{zip}",
            "location": coord_dict,
            "address": f"zipcode {zip}",
            "radius": zipradius,
            "region": "us",
            "type": "convenience_store",
            "key": config.api_key,
            }

    endpoint_url= "https://maps.googleapis.com/maps/api/place/textsearch/json"

    res = requests.get(endpoint_url, params = params)
    results = res.json()
    convenience_store.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        convenience_store.extend(results['results'])
        time.sleep(2)

    shop_name = []
    formatted_address = []
    shop_lat = []
    shop_lng = []
    type = []

    for i in range(len(convenience_store)):
        shop = convenience_store[i]

        try:
            shop_name.append(shop['name'])
        except:
            shop_name.append('none')

        try:
            formatted_address.append(shop['formatted_address'])
        except:
            formatted_address.append('none')

        try:
            shop_lat.append(shop['geometry']['location']['lat'])
        except:
            shop_lat.append('none')

        try:
            shop_lng.append(shop['geometry']['location']['lng'])
        except:
            shop_lng.append('none')
        
        try:
            type.append(shop['types'])
        except:
            type.append('none')

    df_dict = {'shop_name':shop_name, 'shop_address':formatted_address, 'shop_lat':shop_lat, 'shop_lng':shop_lng, 'types':type}

    convenience_store = pd.DataFrame(df_dict)

    convenience_store.to_csv(f'Data/API_queries/convenience_store/convenience_store{zip}.csv')

    return convenience_store


restaurant query function:

In [230]:
def restaurant_query(zip, coord_dict, zipradius):
    restaurants= []

    params = { 
            "query": f"restaurant in {zip}",
            "location": coord_dict,
            "address": f"zipcode {zip}",
            "radius": zipradius,
            "region": "us",
            "type": "restaurant, cafe",
            "key": config.api_key,
            }

    endpoint_url= "https://maps.googleapis.com/maps/api/place/textsearch/json"

    res = requests.get(endpoint_url, params = params)
    results = res.json()
    restaurants.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        restaurants.extend(results['results'])
        time.sleep(2)

    shop_name = []
    formatted_address = []
    shop_lat = []
    shop_lng = []
    type = []

    for i in range(len(restaurants)):
        shop = restaurants[i]

        try:
            shop_name.append(shop['name'])
        except:
            shop_name.append('none')

        try:
            formatted_address.append(shop['formatted_address'])
        except:
            formatted_address.append('none')

        try:
            shop_lat.append(shop['geometry']['location']['lat'])
        except:
            shop_lat.append('none')

        try:
            shop_lng.append(shop['geometry']['location']['lng'])
        except:
            shop_lng.append('none')
        
        try:
            type.append(shop['types'])
        except:
            type.append('none')

    df_dict = {'shop_name':shop_name, 'shop_address':formatted_address, 'shop_lat':shop_lat, 'shop_lng':shop_lng, 'types':type}

    restaurants = pd.DataFrame(df_dict)

    restaurants.to_csv(f'Data/API_queries/restaurant/restaurants{zip}.csv')

    return restaurants


# queries by zipcode

for-loop that will iterate through all zip codes and run all query functions

In [231]:
for zip_code in zip_codes:
    coord = zip_codes[zip_code]["coord"]
    radius = zip_codes[zip_code]["radius"]
    zip_codes[zip_code]["dataframe"] = { "grocery": grocery_query(zip_code, coord, radius),
                                         "gas_station": gasstation_query(zip_code, coord, radius),
                                         "convenience_store": convstore_query(zip_code, coord, radius),
                                         "restaurant": restaurant_query(zip_code, coord, radius),}

# scratch coding area

In [117]:
zip_code = 37216
coord = zip_codes[zip_code]["coord"]
radius = zip_codes[zip_code]["radius"]
zip_codes[zip_code]["dataframe"] = { "grocery": grocery_query(zip_code, coord, radius),
                                     "gas_station": gasstation_query(zip_code, coord, radius),
                                     "convenience_store": convstore_query(zip_code, coord, radius),
                                     "restaurant": restaurant_query(zip_code, coord, radius),}

In [124]:
zip_code = 37216
coord = zip_codes[zip_code]["coord"]
radius = zip_codes[zip_code]["radius"]
zip_codes[zip_code]["dataframe"] = {"junkfood": junkfood_query(zip_code, coord, radius),}

In [128]:
junkfood_query(37216, coord_dict['coord37216'], zipradius['radius37216'])

,shop_name,shop_address,shop_lat,shop_lng,types
0,MAPCO,"4134 Gallatin Pike, Nashville, TN 37216",36.222484,-86.725617,"[gas_station, convenience_store, food, point_of_interest, store, establishment]"
1,Kwik Sak,"3710 Gallatin Pike, Nashville, TN 37216",36.214093,-86.730432,"[convenience_store, gas_station, food, point_of_interest, store, establishment]"
2,Speedway,"2600 Gallatin Pike, Nashville, TN 37216",36.197179,-86.741545,"[gas_station, convenience_store, food, point_of_interest, store, establishment]"
3,Shell,"4800 Gallatin Pike, Nashville, TN 37216",36.231471,-86.724566,"[gas_station, atm, convenience_store, finance, food, point_of_interest, store, establishment]"
4,Mapco,"4126 Gallatin Pike, Nashville, TN 37216",36.222390,-86.725375,"[convenience_store, food, point_of_interest, store, establishment]"
5,Z-Mart,"927 Gallatin Ave, Nashville, TN 37206",36.187961,-86.745941,"[convenience_store, atm, gas_station, finance, food, point_of_interest, store, establishment]"
6,Turkeyhill,"3358 Gallatin Pike, Nashville, TN 37216",36.210004,-86.733515,"[gas_station, point_of_interest, establishment]"
7,Five Star C-Store,"2301 Riverside Dr, Nashville, TN 37216",36.205032,-86.724146,"[convenience_store, food, point_of_interest, store, establishment]"
8,MAPCO Mart,"1201 Gallatin Pike S, Madison, TN 37115",36.247771,-86.720061,"[gas_station, convenience_store, food, point_of_interest, store, establishment]"
9,MAPCO Mart,"1516 Gallatin Pike S, Madison, TN 37115",36.237731,-86.724767,"[gas_station, convenience_store, restaurant, food, point_of_interest, store, establishment]"


# following code is trial and error test coding

following is test query performed on one zipcode, while troubleshooting query results

In [ ]:
#gmaps = googlemaps.Client(key={api_key})

In [ ]:
#not needed i guess ?????????????

#url = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery%20stores%20in%20Nashville%20Tennessee&key={api_key}"

#payload={}
#headers = {}

#response = requests.request('GET', url, headers=headers, data=payload)

In [ ]:
#place types used: grocery_or_supermarket, supermarket, bakery, food

grocery_stores37216 = []
params37216 = {}

#endpoint_url37216 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery+store+in+37216&location=37216&region=us&type=grocery_or_supermarket,supermarket,bakery,food&key={api_key}"

res37216 = requests.get(endpoint_url37216, params = params37216)
results37216 = json.loads(res37216.content)
grocery_stores37216.extend(results37216['results'])
time.sleep(2)
while "next_page_token" in results37216:
    params37216['pagetoken'] = results37216['next_page_token'],
    res37216 = requests.get(endpoint_url37216, params = params37216)
    results37216 = json.loads(res37216.content)
    grocery_stores37216.extend(results37216['results'])
    time.sleep(2)

In [ ]:
res37216

In [ ]:
results37216

In [ ]:
shop_name37216 = []
formatted_address37216 = []
shop_lat37216 = []
shop_lng37216 = []

for i in range(len(grocery_stores37216)):
    shop37216 = grocery_stores37216[i]

    try:
        shop_name37216.append(shop37216['name'])
    except:
        shop_name37216.append('none')

    try:
        formatted_address37216.append(shop37216['formatted_address'])
    except:
        formatted_address37216.append('none')

    try:
        shop_lat37216.append(shop37216['geometry']['location']['lat'])
    except:
        shop_lat37216.append('none')

    try:
        shop_lng37216.append(shop37216['geometry']['location']['lng'])
    except:
        shop_lng37216.append('none')

In [ ]:
df_dict37216 = {'shop_name':shop_name37216, 'shop_address':formatted_address37216, 'shop_lat':shop_lat37216, 'shop_lng':shop_lng37216}

grocery_stores37216_df = pd.DataFrame(df_dict37216)

grocery_stores37216_df['duplicator'] = grocery_stores37216_df['shop_name'] + grocery_stores37216_df['shop_address']

grocery_stores37216_df.drop_duplicates(['duplicator'], inplace=True)



In [ ]:
grocery_stores37216_df

In [ ]:
mixedresultsgrocery_stores37216_df = grocery_stores37216_df

In [ ]:
results37216

In [ ]:
mixedresultsgrocery_stores37216_df

In [ ]:
#original test API query

grocery_stores = []
params1 = {}

#endpoint_url1 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=grocery%20stores%20in%20Nashville%20Tennessee&key={api_key}"
         
res1 = requests.get(endpoint_url1, params = params1)
results1 =  json.loads(res1.content)
grocery_stores.extend(results1['results'])
time.sleep(2)
while "next_page_token" in results1:
     params1['pagetoken'] = results1['next_page_token'],
     res1 = requests.get(endpoint_url1, params = params1)
     results1 = json.loads(res1.content)
     grocery_stores.extend(results1['results'])
     time.sleep(2)

In [ ]:
#original test API query


shop_name1 = []
formatted_address1 = []
shop_lat1 = []
shop_lng1 = []

for i in range(len(grocery_stores)):
    shop = grocery_stores[i]

    try:
        shop_name1.append(shop['name'])
    except:
        shop_name1.append('none')

    try:
        formatted_address1.append(shop['formatted_address'])
    except:
        formatted_address1.append('none')

    try:
        shop_lat1.append(shop['geometry']['location']['lat'])
    except:
        shop_lat1.append('none')

    try:
        shop_lng1.append(shop['geometry']['location']['lng'])
    except:
        shop_lng1.append('none')

In [ ]:
#original test API query


df_dict1 = {'shop_name':shop_name1, 'shop_address':formatted_address1, 'shop_lat':shop_lat1, 'shop_lng':shop_lng1}

grocery_stores_df = pd.DataFrame(df_dict1)
#grocery_stores_df['data_zipcode'] = data_zipcode
grocery_stores_df['duplicator'] = grocery_stores_df['shop_name'] + grocery_stores_df['shop_address']

grocery_stores_df.drop_duplicates(['duplicator'], inplace=True)

#all_grocery_stores = all_grocery_stores.append(grocery_stores_df)

In [ ]:
#restaurants = []
#params2 = {}

#endpoint_url2 = "https://maps.googleapis.com/maps/api/place/textsearch/json?query=fast%20food%20in%20Nashville%20Tennessee&key={api_key}"
         
#res2 = requests.get(endpoint_url2, params = params2)
#results2 =  json.loads(res2.content)
#restaurants.extend(results2['results'])
#time.sleep(2)
#while "next_page_token" in results2:
     #params2['pagetoken'] = results2['next_page_token'],
     #res2 = requests.get(endpoint_url2, params = params2)
     #results2 = json.loads(res2.content)
     #restaurants.extend(results2['results'])
     #time.sleep(2)

In [ ]:
#original test API query


restaurants = []
params2 = {}

#endpoint_url2 = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query=nashville+tn&type=restaurant&key={api_key}'
         
res2 = requests.get(endpoint_url2, params = params2)
results2 =  json.loads(res2.content)
restaurants.extend(results2['results'])
time.sleep(2)
while "next_page_token" in results2:
     params2['pagetoken'] = results2['next_page_token'],
     res2 = requests.get(endpoint_url2, params = params2)
     results2 = json.loads(res2.content)
     restaurants.extend(results2['results'])
     time.sleep(2)

In [ ]:
#original test API query


shop_name2 = []
formatted_address2 = []
shop_lat2 = []
shop_lng2 = []

for i in range(len(restaurants)):
    shop2 = restaurants[i]

    try:
        shop_name2.append(shop2['name'])
    except:
        shop_name2.append('none')

    try:
        formatted_address2.append(shop2['formatted_address'])
    except:
        formatted_address2.append('none')

    try:
        shop_lat2.append(shop2['geometry']['location']['lat'])
    except:
        shop_lat2.append('none')

    try:
        shop_lng2.append(shop2['geometry']['location']['lng'])
    except:
        shop_lng2.append('none')

In [ ]:
#original test API query


df_dict2 = {'shop_name':shop_name2, 'shop_address':formatted_address2, 'shop_lat':shop_lat2, 'shop_lng':shop_lng2}

restaurants_df = pd.DataFrame(df_dict2)
#grocery_stores_df['data_zipcode'] = data_zipcode
restaurants_df['duplicator'] = restaurants_df['shop_name'] + restaurants_df['shop_address']

restaurants_df.drop_duplicates(['duplicator'], inplace=True)

#all_grocery_stores = all_grocery_stores.append(grocery_stores_df)